## Understanding Retrieval-Augmented Generation (RAG) & Use of VectorStore
*by Ong Chin Ann*

#### Objectives
- To understand the concept of Retrival Augmented Generation (RAG)
- To understand the usage of text-embedding model for vectorization
- To familarized with LangChain RAG-related components (Document Loader, Text Splitter, retrievers etc...)
- To develop a custom chatbot with RAG approach


If you find understanding RAG difficult, maybe you can look through the following resource after you have completed the exercise/walkthrough. <br/>
Link: https://www.datacamp.com/blog/what-is-retrieval-augmented-generation-rag


#### RAG Processes
* Step 1: Loading and Chunking Data
* Step 2: Construct Vector Store / Database
* Step 3: Perform Similarity Search based on user Query
* Step 4: Generate response from LLM based on document retrieved from the Vector Database and user query


##### Required Dependencies
- pip install pypdf
- pip install docx2txt


##### Loading required libraries

Link: 
- PDF Loader: https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/
- DOCX Loader: https://python.langchain.com/v0.1/docs/integrations/document_loaders/microsoft_word/
- FAISS: https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/
- AzureOpenAIEmbeddings: https://python.langchain.com/v0.1/docs/integrations/text_embedding/azureopenai/

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader, Docx2txtLoader, PyPDFLoader
load_dotenv(override=True)

True

#### Step 1a: Document Loader
The following code shows how to load document (docx & pdf) using Docx2txtLoader & PyPDFLoader classes from LangChain

Link: 
- PDF Loader: https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/
- DOCX Loader: https://python.langchain.com/v0.1/docs/integrations/document_loaders/microsoft_word/

In [2]:
## Loading PDF Documents
info_loader = PyPDFLoader("resources/CC0003-Outline-AY2021-S28631bdc2-fab3-4039-a923-723ef29f6e8e.pdf")
course_info = info_loader.load()
print("Num Docs", len(course_info))
course_info

Num Docs 1


[Document(page_content='                                            \n \n \nCC000 3 Ethics and Civics in a Multicultural World  \n        \nAcademic Year  AY202 2/23 \nAcademic Units  2AUs  \nTutorial Hours  26 \n \nCOURSE AIMS  \nThis course aims to equip students with the necessary philosophical foundations to understand theories of ethics \nand subsequently apply those theories to real -life scenarios and issues. It also aims to enable students to understand \nand critically assess th e civic institutions that structure their local and global communities. To these ends, the course \nwill examine the nature of ethics, its understanding across different cultures, and how it is manifested in concepts, \nsocial structures, and governance instit utions.  \n \nTopics to be explored include human rights, democracy, freedom of speech  and inequality. The rights and duties of \ncitizenship shall be a unifying theme. Students will think through assumptions they hold on all of these matters. \

In [4]:
## Loading PDF Documents
content_loader = PyPDFLoader("resources/cc3.pdf")
course_content = content_loader.load()
print("Num Docs", len(course_content))
course_content

Num Docs 53


[Document(page_content='Lecture 1:Why do ethics and civics matter to everyone/ what is the purposeof studying this course?-Learn how to engage in a reasoned discussion about issues in ethics and civics(which is what we should do in a multicultural world)-Learn to critically examine the various reasons that all of us may have, given ourdifferences, to believe or act with regard to ethical issues-Different people have different ideas and conceptions as to how they should livewhether as individuals or as citizens-We belong in overlapping societies (e.g. Singapore society, Internet society, schoolsocieties)-Reflecting on the kinds of actions that either would or would not contribute to how wethink we should live, both as an individual and as a citizen, we find that this includesa wide range of concerns-Provide some direction on how we can better come up with and/ or evaluate ourreasons in all these situations and more (these situations do matter to us or others)Convincing others why we sho

#### Step 1b: Document Chunking & Split
When sending a context for LLM for reasoning/inference, its best not to dump in the entire documents as this will cause more tokens to be consume and sometimes can be slow or inaccurate. Hence there is a need to split the document into chunks which will be stored/index using Vector Store/Databases. 

When parsing the context, we only need to find relavent chunks from the vector store/DB and send over to LLM. This will make the response much more accurate and faster. 

Link: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/character_text_splitter/

In [5]:
from langchain_text_splitters import CharacterTextSplitter

text_spliter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=400)
course_info_chuncks = text_spliter.split_documents(course_info)
course_content_chuncks = text_spliter.split_documents(course_content)

print("Course Info Chunks", len(course_info_chuncks))
print("Course Content Chunks", len(course_content_chuncks))



Course Info Chunks 1
Course Content Chunks 53


combine both documents chunks into single collections

In [6]:
allchunks = []

allchunks.extend(course_info_chuncks)
allchunks.extend(course_content_chuncks)

print("N Combined Chunk for both documents: ", len(allchunks))



N Combined Chunk for both documents:  54


#### Step 2: Vector Stores, Databases, & Indexing

Now that the chucks from both documents are ready, we will now create an index and store them into a database called Vector Store/Vector Database (Knowledge Base). This is a special database which uses text embedding model which translate a bunch of text into vector. 

This vector will be useful for search or comparison for similar text given. That's is how the relavent chunk of documents is retrieved when we pass in a query so the vector store/DB could return the relavent chunks of document which will later being passed to LLM as a context/knowledge.

In this exercise, we will be using the Facebook AI Similarity Search (FAISS) as vector store while the AzureOpenAI Text Embedding Model will be use for the text to vector conversion process. Note that FAISS is considered a local database, same goes to ChormaDB while Pinecone and AzureAISearch is considered a cloud-based vector store which uses API calls.

- AzureOpenAI Text Embedding Model: https://python.langchain.com/v0.1/docs/integrations/text_embedding/azureopenai/
- Facebook AI Similarity Search (FAISS) :
    - Vector Database
    - Docs: https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html

*__Acknowledgement__: This access key and resources are supported by NTU EdeX Teaching and Learning Grants 2023-2024 (Call 1)  from the Center for Teaching, Learning & Pedagogy (CTLP) for the project titled "AskNarelle". Please do not share and distribute to other parties*

##### Other Vector Stores/Databases
- ChromaDB
    - Docs: https://python.langchain.com/v0.1/docs/integrations/vectorstores/chroma/
- Pinecone:
    - Docs: https://python.langchain.com/v0.1/docs/integrations/retrievers/self_query/pinecone/
- AzureAISearch :
    - Docs: https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [8]:
from langchain_openai import OpenAIEmbeddings
load_dotenv(override=True)
new_text_embedding =  OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
new_db = FAISS.load_local("resources/db", new_text_embedding, allow_dangerous_deserialization=True)

docs = new_db.similarity_search("What is this course all about?", k=5)

for d in docs:
    print(d)
    print("\n")

page_content='Lecture 1:Why do ethics and civics matter to everyone/ what is the purposeof studying this course?-Learn how to engage in a reasoned discussion about issues in ethics and civics(which is what we should do in a multicultural world)-Learn to critically examine the various reasons that all of us may have, given ourdifferences, to believe or act with regard to ethical issues-Different people have different ideas and conceptions as to how they should livewhether as individuals or as citizens-We belong in overlapping societies (e.g. Singapore society, Internet society, schoolsocieties)-Reflecting on the kinds of actions that either would or would not contribute to how wethink we should live, both as an individual and as a citizen, we find that this includesa wide range of concerns-Provide some direction on how we can better come up with and/ or evaluate ourreasons in all these situations and more (these situations do matter to us or others)Convincing others why we should be all

#### Step 3: Perform Similarity Search based on user Query

Now that you have created a vector database by feeding in the combine chunks for both documents.
You have also generated a retrieval object which will return top 3 most relavent chunks of document based on a query. 
Now, its time to verify the retrieval and try to search relavent chunk of documents based on query given.

In [9]:
query = "What is the course all about?"
result = new_db.similarity_search(query, k=3)

print("Total chunck returned: ", len(result))
print(result[0])


print("\n\n=====================================================================")
for r in result:
    print(r.page_content)
    print("  ### [Source Document: ", r.metadata['source'], "]")
    print("\n     ------------------------------------        \n")
print("=====================================================================")




Total chunck returned:  3
page_content='Lecture 1:Why do ethics and civics matter to everyone/ what is the purposeof studying this course?-Learn how to engage in a reasoned discussion about issues in ethics and civics(which is what we should do in a multicultural world)-Learn to critically examine the various reasons that all of us may have, given ourdifferences, to believe or act with regard to ethical issues-Different people have different ideas and conceptions as to how they should livewhether as individuals or as citizens-We belong in overlapping societies (e.g. Singapore society, Internet society, schoolsocieties)-Reflecting on the kinds of actions that either would or would not contribute to how wethink we should live, both as an individual and as a citizen, we find that this includesa wide range of concerns-Provide some direction on how we can better come up with and/ or evaluate ourreasons in all these situations and more (these situations do matter to us or others)Convincing o

#### Step 4: Generate response from LLM based on document retrieved and query

This is the last portion where will will construct the logic of the chatbot. The procedure and flow will be as follow:
1) Construct a LLM and system prompt
2) get user query.
3) the retrieval will return relavent chunks of documents based on query given.
4) construct the relavent chunks as the context along with user query.
5) pass the relavent chunk and user query to LLM for inferent
6) display the response and append into chatlog along with query.

In [10]:
from time import sleep
import os
import langchain
from langchain.llms import AzureOpenAI                                          ## This object is a connector/wrapper for OpenAI LLM engine
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage      ## These are the commonly used chat messages

load_dotenv(override=True)

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [11]:
persona = "You are a teaching assistant at for the course SC1015 at NTU."
task ="your task is to answer student query about the data science and ai course."
context = "the context will be provided based on the course information and FAQ along with the user query"
condition = "If user ask any query beyond data science and ai, tell the user you are not an expert of the topic the user is asking and say sorry. If you are unsure about certain query, say sorry and advise the user to contact the instructor at instructor@ntu.edu.sg"
### any other things to add on

## Constructing initial system message
sysmsg = f"{persona} {task} {context} {condition}"
chatlog = [SystemMessage(content=sysmsg)]

##### Define a function to perform the retrival and consolidation of the chunks for easy processing

In [12]:
def search_chunks(query):
    search_result = new_db.similarity_search(query, k=3)
    context = []
    for r in search_result:
        context.append(r.page_content)

    instruction = "try to understand the userquery and answer based on the context given below:\n"
    return SystemMessage(content=f"{instruction}'context':{context}, 'userquery':{query}")

In [13]:
x = search_chunks("what is the course all about?")
print(x.content)

try to understand the userquery and answer based on the context given below:
'context':['Lecture 1:Why do ethics and civics matter to everyone/ what is the purposeof studying this course?-Learn how to engage in a reasoned discussion about issues in ethics and civics(which is what we should do in a multicultural world)-Learn to critically examine the various reasons that all of us may have, given ourdifferences, to believe or act with regard to ethical issues-Different people have different ideas and conceptions as to how they should livewhether as individuals or as citizens-We belong in overlapping societies (e.g. Singapore society, Internet society, schoolsocieties)-Reflecting on the kinds of actions that either would or would not contribute to how wethink we should live, both as an individual and as a citizen, we find that this includesa wide range of concerns-Provide some direction on how we can better come up with and/ or evaluate ourreasons in all these situations and more (these 

In [14]:
query = input("Enter your message: ")
usermsg = HumanMessage(content=query)


while query != "exit":
    print(f"Human : {query}\n")
    chatlog.append(usermsg)

    context = search_chunks(query)
    templog = chatlog + [context]
    response = llm.invoke(templog)
    sleep(2)
    print(f"AI    : {response.content}\n")
    chatlog.append(response)

        
    query = input("Enter your message: ")
    usermsg = HumanMessage(content=query)

Human : 

AI    : Sorry, I am not able to provide a relevant response to the user query based on the context provided. If you have any questions related to Data Science and AI, I would be happy to help you.

Human : what is this course about

AI    : This course is about Ethics and Civics in a Multicultural World. It aims to provide students with the philosophical foundations to understand ethical theories and apply them to real-life scenarios and issues. Topics covered include human rights, democracy, freedom of speech, inequality, and civic institutions. The course also encourages students to critically assess civic structures and institutions, make well-informed arguments on contemporary issues, and apply ethics and civics concepts to the Singapore context. By the end of the course, students are expected to identify morally relevant features of situations, explain their moral responsibilities in local and global communities, and critically assess civic structures and institutions.



Also, if you recall, we specify the `chunk_size` and `chunk_overlapping` on `text_spliter` object, these are some parameter we can tune to get a better retrieval. Also the `top k` can be adjust to provide `llm` sufficient knowledge to infer user query along with creative and effective prompt design and engineering.


Extra Readings:
- https://learn.microsoft.com/en-us/azure/search/retrieval-augmented-generation-overview